# Step 2
## Match the gradients to working memory task time-series and predict task condition

In this step we take use the three non-trivial gradients with highest eigenvalues and match them to task time-series. This will allow us to re-express the task-related BOLD time-series as a time series of gradients. The steps are the following:

1. Load single run of task time-series.
2. Normalize the task time-series to have mean = 0 and variance = 1 (z-score).
3. For each volume independently, compute coefficients of a linear regression model with gradients as predictors.

To reproduce the results, you need to provide your own copy of HCP working memory task data.  

In [1]:
import glob
import h5py as h5
import time
import os

import nibabel as nib
import numpy as np
from fgrad.regress import regress_subject, trim_data

/usr/local/lib/python2.7/site-packages/nibabel/cifti2/parse_cifti2.py:24: FutureWarning: We no longer carry a copy of the 'six' package in nibabel; Please import the 'six' package directly
  from ..externals.six import BytesIO


In [2]:
gradients = nib.load('../data/rsFC_eigenvectors.dscalar.nii').get_data().T

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [5]:
subjects = sorted(glob.glob('/Users/marcel/projects/HCP/data/*'))
n_sub = len(subjects)
n_runs = 2 # maximum number of runs per subject
n_grad = 10 # number of gradients to fit
n_tp = 405 # maximum number of time points in time series

ts_gradients = np.zeros([n_sub, n_runs, n_tp, n_grad])

In [6]:
for j, s in enumerate(subjects):
    print "Subject %s, %d of %d" % (s, j+1, n_sub)
    files = sorted(glob.glob('%s/MNINonLinear/Results/tfMRI_WM_*/tfMRI_*_Atlas_MSMAll.dtseries.nii' % s))
    print "Found %d runs" % len(files)
    if len(files) > 0:
        for i, f in enumerate(files):
            t = time.time()
            ts_gradients[j, i, :, :] = regress_subject(files[0], gradients[:,0:n_grad])
            print "Run %d/%d took %.2f seconds" % (i+1, len(files), time.time() - t)
    else:
        print "Found no runs for subject %s!" % s

ts_gradients, r, subjects = trim_data(ts_gradients, subjects)

Subject /Users/marcel/projects/HCP/data/100307, 1 of 2
Found 2 runs


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Run 1/2 took 5.13 seconds


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Run 2/2 took 5.16 seconds
Subject /Users/marcel/projects/HCP/data/666, 2 of 2
Found 0 runs
Found no runs for subject /Users/marcel/projects/HCP/data/666!


In [9]:
if os.path.isfile('../data/reconstructed_WM.hdf5'):
    os.remove('../data/reconstructed_WM.hdf5')

f = h5.File('../data/reconstructed_WM.hdf5')
g = f.create_group('Working_memory')
g.create_dataset('LR', data = ts_gradients[:,0,:,:], compression = "gzip", chunks = (1,n_tp,n_grad))
g.create_dataset('RL', data = ts_gradients[:,1,:,:], compression = "gzip", chunks = (1,n_tp,n_grad))
g.create_dataset('Subjects', data = s)

f.close()